In [17]:
# LOAD DATA

import os, csv
import pandas as pd
from pyspark.sql import SparkSession, functions, types
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.ml.feature import StringIndexer, VectorAssembler, SQLTransformer
from pyspark.ml.regression import DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel('WARN')

# Read in data. We need to read in the first row of the CSV 
# to get the headers so we can dynamically create the schema.
DATA_DIR = 'project_data'
file_name = 'data_windows_05s.txt'

with open(os.path.join(DATA_DIR, file_name)) as f:
    reader = csv.reader(f, delimiter="\t")
    row1 = next(reader)

# Dynamically create schema. All cols except filename are numbers.
schema = types.StructType(
        [types.StructField(field_name, types.FloatType()) if field_name != 'filename' \
            else types.StructField(field_name, types.StringType()) for field_name in row1])

df = spark.read.csv(os.path.join(DATA_DIR, file_name), sep='\t', header=True, schema=schema)

# confirm the labels are right
labels = df.select('label').collect()
falls = [l for l in labels if l[0] == 1]
non_falls = [l for l in labels if l[0] == 0]
print("Falls: {}, Non-falls: {}".format(len(falls), len(non_falls)))

# Get all columns except trial_id and label as features
features = [col for col in df.columns if col not in ['trial', 'label', 'filename']]
train, validation = df.randomSplit([0.75, 0.25])

Falls: 6300, Non-falls: 12290


In [18]:
## LOGISTIC REGRESSION

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import HashingTF, Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

# Hyperparameters
numFolds = 5
epochs = 100

# Assemble Pipeline
assembler = VectorAssembler(
    inputCols=features,
    outputCol='features')
classifier = LogisticRegression(maxIter=epochs)
pipeline = Pipeline(stages=[assembler, classifier])

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [300]) \
    .addGrid(classifier.regParam, [0]) \
    .build()

cv = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName='weightedPrecision'),
                          numFolds=numFolds)  
model = cv.fit(train)
predictions = model.transform(validation)
evaluator = BinaryClassificationEvaluator()

evaluator = MulticlassClassificationEvaluator()
print("Weighted Precision: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})))
print("Weighted Recall: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})))
print("f1: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "f1"})))
print("Accuracy: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))

Weighted Precision: 0.985540093833607
Weighted Recall: 0.9855442176870748
f1: 0.9855419567379619
Accuracy: 0.9855442176870748


In [16]:
# NAIVE BAYES

import numpy as np 
from sklearn.naive_bayes import GaussianNB

cols = [col for col in df.columns]
df_nb = df.select(cols)

df_positive = df_nb.where(df_nb.label == 1).toPandas()
df_negative = df_nb.where(df_nb.label == 0).toPandas()

pos_rows = len(df_positive)
i_pos = int(0.8 * pos_rows)

neg_rows = len(df_negative)
i_neg = int(0.8 * neg_rows)

train_X_pos = df_positive[:i_pos].to_numpy()[:, 3:].astype(np.float32)
test_X_pos = df_positive[i_pos:].to_numpy()[:, 3:].astype(np.float32)
train_y_pos = df_positive['label'][:i_pos].to_numpy().astype(np.float32)
test_y_pos = df_positive['label'][i_pos:].to_numpy().astype(np.float32)

train_X_neg = df_negative[:i_neg].to_numpy()[:, 3:].astype(np.float32)
test_X_neg = df_negative[i_neg:].to_numpy()[:, 3:].astype(np.float32)
train_y_neg = df_negative['label'][:i_neg].to_numpy().astype(np.float32)
test_y_neg = df_negative['label'][i_neg:].to_numpy().astype(np.float32)

train_X = np.concatenate((train_X_pos, train_X_neg))
test_X = np.concatenate((test_X_pos, test_X_neg))
train_y = np.concatenate((train_y_pos, train_y_neg))
test_y = np.concatenate((test_y_pos, test_y_neg))

gnb = GaussianNB()
model = gnb.fit(train_X, train_y)
y_pred = model.predict(train_X)

def calculateAccuracy(actual, prediction):
    errors = np.sum((prediction-actual)**2)
    total = len(labels)
    percentCorrect = 1-errors/total
    return percentCorrect

test_pred = model.predict(test_X)
test_actual = test_y

d = {'prediction': test_pred, 'actual': test_actual}
results = pd.DataFrame(data=d)

true_positive = results.loc[results['prediction'] == 1].loc[results['actual'] == 1].count().to_numpy()[0]
false_positive = results.loc[results['prediction'] == 1].loc[results['actual'] == 0].count().to_numpy()[0]
true_negative = results.loc[results['prediction'] == 0].loc[results['actual'] == 0].count().to_numpy()[0]
false_negative = results.loc[results['prediction'] == 0].loc[results['actual'] == 1].count().to_numpy()[0]

true_positive_rate = true_positive/(true_positive + false_positive)
false_positive_rate = false_positive/(true_positive + false_positive)
true_negative_rate = true_negative/(true_negative + false_negative)
false_negative_rate = false_negative/(true_negative + false_negative)
accuracy = (true_positive + true_negative)/(true_positive + false_positive + true_negative + false_negative)
recall = true_positive/(true_positive + false_negative)
precision = true_positive/(true_positive + false_positive)
f1 = 2*(precision*recall)/(precision+recall)

print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F1: " + str(f1))
print("Accuracy: " + str(accuracy))

Precision: 0.7762237762237763
Recall: 0.44047619047619047
F1: 0.5620253164556962
Accuracy: 0.7674731182795699
